# Determining the Market Value of Debt

If we cannot get access to data on the individual debt instruments issued by the company, then all we can do is use the book value of debt from the balance sheet as an estimate of the market value of debt.

If we still don't have data on the individual debt instruments, but do have an average maturity of the debt, we can do a bit better job with estimating the market value of debt.

If we have data on all the individual debt instruments, the combined with an estimate of the cost of debt, we can get an accurate estimate the market value of debt to be used in the WACC calculation.

This example is divided into three sections, depending on what data are available.

# Some Setup

These inputs will be used throughout.

In [1]:
total_book_debt = 1000000
interest_expense = 60000
cost_of_debt = 0.08

# MV Debt with No Additional Data

Since we don't have access to an average maturity or the individual debt instruments, all we can do is just use the book value of debt.

In [2]:
mv_debt = total_book_debt
mv_debt

1000000

And we are done. It's not a very good estimate though.

# MV Debt with Average Maturity

Let's just say we also knew that the average maturity of the debt was 5 years. You may be able to find this info when the individual debt instruments are not available. The general approach is then to treat the entire total book debt from the balance sheet as a single coupon bond, and determine the value of that coupon bond.

In [3]:
average_maturity = 5

## Determine the Coupon Payment on the Hypothetical Bond

The coupon payment on this hypothetical bond can be estimated by taking the interest expense divided by the total book debt. 

In [4]:
coupon_rate = interest_expense / total_book_debt
coupon_rate

0.06

Now that we have a coupon rate, we can form the bond cash flows. It should be the coupon rate multiplied by principal for each period up until the final period, which is (1 + coupon rate) multiplied by principal.

## Calculate Hypothetical Bond Cash Flows

In [5]:
principal = total_book_debt
coupon_payment = coupon_rate * principal
cash_flows = [coupon_rate * principal for i in range(average_maturity - 1)] + [(1 + coupon_rate) * principal]
cash_flows

[60000.0, 60000.0, 60000.0, 60000.0, 1060000.0]

## Calculate Value of Hypothetical Bond as MV of Debt

Now we can calculate the value of this bond by taking the NPV with the cost of debt. Recall that `numpy`'s NPV function takes the first cash flow as year 0, so we will need to add a 0 at the beginning.

In [6]:
import numpy as np

mv_debt = np.npv(cost_of_debt, [0] + cash_flows)
mv_debt

/home/nick/.local/share/virtualenvs/fin-model-course-eIFMSc8A/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: numpy.npv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.npv instead (https://pypi.org/project/numpy-financial/).
  This is separate from the ipykernel package so we can avoid doing imports until


920145.7992584379

Therefore we have \\$9.2 million as our estimate for the market value of debt.

## Complete the Preceding Steps with One Forumla

We can wrap all the prior steps into one formula, which calculates the value of an annuity with an additional terminal payment. The formula is: $$V = C (\frac{1 - (1 + r_d)^{-t}}{r_d}) + \frac{P}{(1 + r_d)^t} $$

In [7]:
mv_debt = coupon_payment * ((1 - (1 + cost_of_debt)**(-average_maturity))/cost_of_debt) + principal/(1 + cost_of_debt)**average_maturity
mv_debt

920145.7992584382

We can wrap this all up in a function.

In [8]:
def mv_debt_by_average_maturity(average_maturity, cost_of_debt, total_book_debt, interest_expense):
    """
    Calculate the market value of debt based off financial statement data, cost of debt, and average maturity.
    """
    coupon_rate = interest_expense / total_book_debt
    principal = total_book_debt
    coupon_payment = coupon_rate * principal
    
    return coupon_payment * ((1 - (1 + cost_of_debt)**(-average_maturity))/cost_of_debt) + principal/(1 + cost_of_debt)**average_maturity

mv_debt_by_average_maturity(average_maturity, cost_of_debt, total_book_debt, interest_expense)

920145.7992584382

This formula is especially useful when the average maturity is not a whole number of years, say 5.5 years.

In [9]:
mv_debt_by_average_maturity(5.5, cost_of_debt, total_book_debt, interest_expense)

913722.8716722389

# MV Debt with Individual Debt Instruments

The basic process is to value each debt instrument based on its coupon rate and the overall cost of debt, similar to what we did for the hypothetical bond in the last section. Then we sum of the values of all the instruments to find the total market value of debt.

Please note that you must have data on all the debt instruments to use this approach. If any are missing, then it is better to determine the average maturity from the ones you have, and use the approach described in the last section. 

Another approach for missing some instruments is to calculate the ratio of the total instrument principal to total balance sheet debt, then after determining the total market value of the instruments, divide that by the ratio to get an estimate of the market value of debt. But we will not explore this option here.

Here I will load in some data which has information on the individual debt instruments for this company.

In [10]:
import pandas as pd

df = pd.read_excel('debt data.xlsx')
df

,Principal,Coupon Rate,Maturity (years),Maturity (Date)
0,300000,0.07,2,2021-11-06
1,500000,0.05,3,2022-11-06
2,200000,0.06,4,2023-11-06


## Checking if the Approach will be Valid

Let's see if the total of the principal on the instruments matches up to the firm's total debt.

In [11]:
df['Principal'].sum()

1000000

In [12]:
total_book_debt

1000000

We can see these two are the same, so we are fine to go ahead with this approach. For a real company, it will likely never match exactly. If you're close (within a couple percent) then this approach should be fine.

## Calculating the Value of a Debt Instrument

Here we will follow a similar valuation approach to the last section. Since we need to do this for every bond, we will create a function which does this for a single bond and apply it to all the bonds. Let's create this function before working with the `DataFrame`. Here we use the exact same logic from the last section to get the cash flows.

In [13]:
principal = 300000
coupon_rate = 0.07
maturity = 2

coupon_payment = coupon_rate * principal
cash_flows = [coupon_rate * principal for i in range(maturity - 1)] + [(1 + coupon_rate) * principal]
cash_flows

[21000.000000000004, 321000.0]

And get the value in the same way as the last section.

In [14]:
mv_debt = np.npv(cost_of_debt, [0] + cash_flows)
mv_debt

/home/nick/.local/share/virtualenvs/fin-model-course-eIFMSc8A/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: numpy.npv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.npv instead (https://pypi.org/project/numpy-financial/).
  """Entry point for launching an IPython kernel.


294650.2057613168

Let's wrap this up in one function.

In [15]:
def mv_bond(principal, coupon_rate, maturity, cost_of_debt):
    """
    Calculate the market value of bond
    """
    coupon_payment = coupon_rate * principal
    cash_flows = [coupon_payment for i in range(maturity - 1)] + [coupon_payment + principal]
    return np.npv(cost_of_debt, [0] + cash_flows)

mv_bond(principal, coupon_rate, maturity, cost_of_debt)

/home/nick/.local/share/virtualenvs/fin-model-course-eIFMSc8A/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: numpy.npv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.npv instead (https://pypi.org/project/numpy-financial/).
  import sys


294650.2057613168

### A Quick Aside to `DataFrame.apply`

Here we have just three securities, it would be pretty easy to manually call this on each of them. But what if there are 50 securities? You want an automated approach to apply this function to all the rows in the `DataFrame`. This is where we bring in `.apply` on the `DataFrame`. When you do `.apply(function, axis=1)` on a `DataFrame`, it will take each row of the `DataFrame` (the row is now a `Series`) and pass it to the function. Then it will take the result of the function call and put it into a new `Series`.

Let's see how this works with a simple example that just prints out what we're getting from the `.apply` call.

In [16]:
def understand_apply(series):
    """
    For demonstration purposes to understand DataFrame.apply
    """
    print('Calling understand_apply')
    print(f'Got values:\n{series}\n')
    print(f'Values have type {type(series)}')
    principal = series['Principal']
    print(f'Value of principal in the series: {principal}')
    print('\n\n\n')  # separate output

df.apply(understand_apply, axis=1)

Calling understand_apply
Got values:
Principal                        300000
Coupon Rate                        0.07
Maturity (years)                      2
Maturity (Date)     2021-11-06 00:00:00
Name: 0, dtype: object

Values have type <class 'pandas.core.series.Series'>
Value of principal in the series: 300000




Calling understand_apply
Got values:
Principal                        500000
Coupon Rate                        0.05
Maturity (years)                      3
Maturity (Date)     2022-11-06 00:00:00
Name: 1, dtype: object

Values have type <class 'pandas.core.series.Series'>
Value of principal in the series: 500000




Calling understand_apply
Got values:
Principal                        200000
Coupon Rate                        0.06
Maturity (years)                      4
Maturity (Date)     2023-11-06 00:00:00
Name: 2, dtype: object

Values have type <class 'pandas.core.series.Series'>
Value of principal in the series: 200000






0    None
1    None
2    None
dtype: object

## Calculating the Value of All the Debt Instruments

Now that we understand `.apply`, we can use it to apply this `mv_bond` function to all the `DataFrame` rows. But we'll need to make a new function based off the `mv_bond` function which can take a `Series` rather than the individual values.

In [17]:
def mv_bond_for_apply(series, cost_of_debt):
    """
    Calculate market value of a bond. This version to be used with DataFrame.apply
    """
    principal = series['Principal']
    coupon_rate = series['Coupon Rate']
    maturity = series['Maturity (years)']
    return mv_bond(principal, coupon_rate, maturity, cost_of_debt)

df.apply(mv_bond_for_apply, axis=1, cost_of_debt=cost_of_debt)

/home/nick/.local/share/virtualenvs/fin-model-course-eIFMSc8A/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: numpy.npv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.npv instead (https://pypi.org/project/numpy-financial/).
  import sys


0    294650.205761
1    461343.545191
2    186751.492640
dtype: float64

You can now see we get the value of each bond in a `Series`. The last thing to do is assign that back to the `DataFrame` as a column.

In [18]:
df['Value'] = df.apply(mv_bond_for_apply, axis=1, cost_of_debt=cost_of_debt)
df

/home/nick/.local/share/virtualenvs/fin-model-course-eIFMSc8A/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: numpy.npv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.npv instead (https://pypi.org/project/numpy-financial/).
  import sys


,Principal,Coupon Rate,Maturity (years),Maturity (Date),Value
0,300000,0.07,2,2021-11-06,294650.205761
1,500000,0.05,3,2022-11-06,461343.545191
2,200000,0.06,4,2023-11-06,186751.492640


Now we can simply take the sum of those values to get the market value of debt for the firm.

In [19]:
mv_debt = df['Value'].sum()
mv_debt

942745.2435924213

## Using Dates Instead of Years to Maturity

Usually with individual debt instruments you will not get a number of years remaining until maturity, but instead a maturity date, and you need to calculate the years to maturity.

When using this approach, the annuity formula should be used to handle non-integer years. Let's first swap that out and make sure it works the same.

In [20]:
def mv_bond_annuity_approach(principal, coupon_rate, maturity, cost_of_debt):
    """
    Calculate the market value of bond with non-integer maturity
    """
    coupon_payment = coupon_rate * principal
    return coupon_payment * ((1 - (1 + cost_of_debt)**(-maturity))/cost_of_debt) + principal/(1 + cost_of_debt)**maturity


def mv_bond_annuity_approach_for_apply(series, cost_of_debt):
    """
    Calculate market value of a bond with non-integer maturity. This version to be used with DataFrame.apply
    """
    principal = series['Principal']
    coupon_rate = series['Coupon Rate']
    maturity = series['Maturity (years)']
    return mv_bond_annuity_approach(principal, coupon_rate, maturity, cost_of_debt)

df.apply(mv_bond_annuity_approach_for_apply, axis=1, cost_of_debt=cost_of_debt)

0    294650.205761
1    461343.545191
2    186751.492640
dtype: float64

We see we get the same results. Now let's go to calculating the maturities. First, so we're not cheating, let's drop the years to maturity and the value we already calculated.

In [21]:
df = df.drop(['Maturity (years)', 'Value'], axis=1)  # axis=1 here means column, the command can also be used to drop rows with axis=0
df

,Principal,Coupon Rate,Maturity (Date)
0,300000,0.07,2021-11-06
1,500000,0.05,2022-11-06
2,200000,0.06,2023-11-06


### Setting the Data Type Correctly

Depending on your data, `pandas` may or may not know where your dates are. You can check this by looking at the data types (`dtypes`) of the columns:

In [22]:
df.dtypes

Principal                   int64
Coupon Rate               float64
Maturity (Date)    datetime64[ns]
dtype: object

We can see here the data type for maturity is listed as `datetime64[ns]`, which means it's a date, which is what we want. If it instead was coming as `object`, then we would have to run the following command:

In [23]:
df['Maturity (Date)'] = pd.to_datetime(df['Maturity (Date)'])

In [24]:
df

,Principal,Coupon Rate,Maturity (Date)
0,300000,0.07,2021-11-06
1,500000,0.05,2022-11-06
2,200000,0.06,2023-11-06


In [25]:
df.dtypes

Principal                   int64
Coupon Rate               float64
Maturity (Date)    datetime64[ns]
dtype: object

As we already had it as a date, it had no effect, but it didn't hurt.

### Calculating the Difference Between the Maturity Date and Today

Let's first pull a single date from the `DataFrame` to work with.

In [26]:
date = df.loc[0, 'Maturity (Date)']  # get the first row (0-indexed) value from the Maturity (Date) column
date

Timestamp('2021-11-06 00:00:00')

We can see this value shows as a `Timestamp`, which is the way `pandas` represents dates.

Next we need to get the value for today. We can use the built-in `datetime` module for this.

In [27]:
import datetime
today = datetime.datetime.today()
today

datetime.datetime(2020, 3, 30, 18, 11, 30, 188646)

This is a `datetime.datetime` object, which works just fine with `pandas`' `Timestamp`. We can directly do math with them:

In [28]:
diff = date - today
diff

Timedelta('585 days 05:48:29.811354')

The difference between two datetime objects is a `Timedelta` object. Now to get a number of years (or any time horizon), we convert it into the total number of seconds, then convert the seconds into years.

In [29]:
seconds = diff.total_seconds()
seconds

50564909.811354

In [30]:
seconds_per_year = 60 * 60 * 24 * 365
seconds_per_year

31536000

In [31]:
years_elapsed = seconds / seconds_per_year
years_elapsed

1.6034027717958523

It may be a little bit off due to leap years. You can certainly adjust for this, but this is usually "good enough" and handling leap years is outside the scope of the class.

Let's create a function for this.

In [32]:
def years_until_from_date(date):
    """
    Calculate the number of years until a date, starting from today.
    """
    today = datetime.datetime.today()
    diff = date - today
    seconds = diff.total_seconds()
    seconds_per_year = 60 * 60 * 24 * 365
    years_elapsed = seconds / seconds_per_year
    return years_elapsed

years_until_from_date(date)

1.6034027682953134

You'll notice it's slightly different, because some seconds have elapsed since running the prior commands! Now let's create the version to use with `DataFrame.apply`.

In [33]:
def years_until_from_date_for_apply(series, date_col='Maturity (Date)'):
    date = series[date_col]
    return years_until_from_date(date)

df.apply(years_until_from_date_for_apply, axis=1)

0    1.603403
1    2.603403
2    3.603403
dtype: float64

Now to wrap it up we can assign back to the `DataFrame`.

In [34]:
df['Maturity (years)'] = df.apply(years_until_from_date_for_apply, axis=1)
df

,Principal,Coupon Rate,Maturity (Date),Maturity (years)
0,300000,0.07,2021-11-06,1.603403
1,500000,0.05,2022-11-06,2.603403
2,200000,0.06,2023-11-06,3.603403


Which now allows us to calculate the value, this time using our calculated maturity.

In [35]:
df['Value'] = df.apply(mv_bond_annuity_approach_for_apply, axis=1, cost_of_debt=cost_of_debt)
df

,Principal,Coupon Rate,Maturity (Date),Maturity (years),Value
0,300000,0.07,2021-11-06,1.603403,295646.641081
1,500000,0.05,2022-11-06,2.603403,465956.671673
2,200000,0.06,2023-11-06,3.603403,187890.536216


And again, we can simply take the sum of those values to get the market value of debt for the firm.

In [36]:
mv_debt = df['Value'].sum()
mv_debt

949493.848970559